In [1]:
import cvxpy as cp
import torch
from cvxpylayers.torch import CvxpyLayer

In [2]:
m , n = 3 , 4
x = cp. Variable ((n , 1) )
F = cp. Parameter (( m , n ) )
g = cp. Parameter (( m , 1) )
lambd = cp.Parameter(1 , 1) , nonneg = True )

In [3]:
objective_fn = cp.norm(F @ x - g) + lambd*cp.norm(x)
constraints = [x >= 0]
problem = cp.Problem(cp.Minimize(objective_fn), constraints)

In [4]:
problem.is_dcp()

True

In [5]:
problem.is_dpp()

True

## Get cone problem data

In [6]:
# converts problem into a cone program
data, _, _ = problem.get_problem_data(solver=cp.SCS)

In [7]:
A = data['A']
b = data['b']
c = data['c']

In [8]:
A.todense()

matrix([[ 0.,  0., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -1.],
        [-1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0., nan, nan, nan, nan],
        [ 0.,  0., nan, nan, nan, nan],
        [ 0.,  0., nan, nan, nan, nan],
        [ 0., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -1.]])

In [9]:
b

array([ 0.,  0.,  0.,  0.,  0., nan, nan, nan,  0.,  0.,  0.,  0.,  0.])

In [10]:
c

array([ 1., nan,  0.,  0.,  0.,  0.])

## Adding layers

In [84]:
F_t = torch . randn (m , n , requires_grad = True )
g_t = torch . randn (m , 1 , requires_grad = True )
lambd_t = torch.rand (1 , 1 , requires_grad = True )

In [85]:
# canonicalizes problem to extract C and R
layer = CvxpyLayer(problem, parameters=[F,g,lambd], variables=[x])

In [96]:
layer.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict(),
 'param_order': [Parameter((20, 10)),
  Parameter((20, 1)),
  Parameter((1, 1), nonneg=True)],
 'param_ids': [247, 248, 250],
 'variables': [Variable((10, 1))],
 'var_dict': {246},
 'compiler': <cvxpy.reductions.dcp2cone.cone_matrix_stuffing.ParamConeProg at 0x7fc0071e99e8>,
 'cone_dims': {'f': 0, 'l': 10, 'q': [21, 11], 'ep': 0, 's': []},
 'info': {'canon_time': 0.005506992340087891,
  'solve_time': 0.003384828567504883}}

In [87]:
# apply map R ◦ s ◦ C, return a solution to the problem
x_star, = layer(F_t,g_t,lambd_t)

In [72]:
# compute the gradient of summing x_star w.r.t  F_t, g_t, lambd_t
x_star.sum().backward()